In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import torch
from torch.utils.data import TensorDataset,DataLoader
import numpy as np
import os
from scipy import stats

os.chdir(os.getcwd())
path = 'data/'
fieldname = '_19790101-20190228.npy'
x1_arr = np.load(path+'z1000'+fieldname) # geopotential height data (9*9 resolution)
x2_arr = np.load(path+'pv300'+fieldname) # potential vorticity data (9*9 resolution)
x3_arr = np.load(path+'z1000'+fieldname) # geopotential height data (9*9 resolution)
print(x1_arr.shape)

x1_arr_flat = stats.zscore(x1_arr.reshape([x1_arr.shape[0],x1_arr.shape[1]*x1_arr.shape[2]])) # normalize and flatten
print(x1_arr_flat.shape)
x2_arr_flat = stats.zscore(x2_arr.reshape([x2_arr.shape[0],x2_arr.shape[1]*x2_arr.shape[2]]))
x3_arr_flat = stats.zscore(x3_arr.reshape([x3_arr.shape[0],x3_arr.shape[1]*x3_arr.shape[2]]))
y_arr = np.load(path+'rain_basin_19790101-20190228.npy') # rain data

tensor_x = torch.Tensor(np.concatenate([x1_arr_flat,x2_arr_flat,x3_arr_flat],axis=1)) # join z and pv data
print(tensor_x.shape)
tensor_y = torch.Tensor(y_arr)

forecast_dataset = TensorDataset(tensor_x,tensor_y) # creates a dataset based on tensors
forecast_dataset2 = []


for j in forecast_dataset:
  forecast_dataset2.append( ( j[0].reshape((3, 9, 9)) , j[1] ) ) # Rotem: ask about order of 162
training_ds, validation_ds = torch.utils.data.random_split(forecast_dataset2, [2195,1464])
training_dataloader = DataLoader(training_ds,batch_size=200,shuffle=True)
valid_dataloader = DataLoader(validation_ds,batch_size=200)



(3659, 9, 9)
(3659, 81)
torch.Size([3659, 243])


AttributeError: 'list' object has no attribute 'shape'

In [31]:
for batch_x,batch_y in training_dataloader:
  print(batch_x.shape, batch_y.shape)

torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([195, 3, 9, 9]) torch.Size([195, 4])


In [35]:
from model2 import Net
import torch.nn as nn
import torch.optim as optim


net = Net()

for x,y in training_dataloader:
    print(x.shape,net(x).shape,y.shape)
    break
    


loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001) 

def compute_accuracy_and_loss(dataloader,net):
    total = 0
    correct = 0
    
    loss = 0
    
    if torch.cuda.is_available():
        net.cuda()
    net.eval()
        
    n_batches = 0
    with torch.no_grad():
        for x,y in dataloader:
            n_batches+=1
            
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()
            pred = net(x)
            
            loss+= loss_func(pred,y).item()
            
            pred = torch.argmax(pred,dim=1)

            correct+=len(torch.where(pred==y)[0])
            total+=len(y)
    loss = loss/n_batches      
    return correct/total, loss

compute_accuracy_and_loss(training_dataloader,net)

RuntimeError: Given input size: (512x1x1). Calculated output size: (512x0x0). Output size is too small

In [33]:
from tqdm import tqdm
if torch.cuda.is_available():
    net.cuda()
    
n_epochs = 50


training_loss_vs_epoch = []
validation_loss_vs_epoch = []

training_acc_vs_epoch = []
validation_acc_vs_epoch = []

pbar = tqdm( range(n_epochs) )

for epoch in pbar:
    
    if len(validation_loss_vs_epoch) > 1:
        pbar.set_description('val acc:'+'{0:.5f}'.format(validation_acc_vs_epoch[-1])+
                             ', train acc:'+'{0:.5f}'.format(training_acc_vs_epoch[-1]))
    
    net.train() # put the net into "training mode"
    for x,y in training_dataloader:
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()
            
        # add the basic training loop here
        optimizer.zero_grad()
        
        output = net(x)
        print(y.shape)
        print(output.shape)
        loss = loss_func(output,y)
        
        loss.backward()
        optimizer.step()
    net.eval() #put the net into evaluation mode
    
    
    train_acc, train_loss = compute_accuracy_and_loss(training_dataloader,net)
    valid_acc, valid_loss =  compute_accuracy_and_loss(valid_dataloader,net)
         
    training_loss_vs_epoch.append( train_loss)    
    training_acc_vs_epoch.append( train_acc )
    
    validation_acc_vs_epoch.append(valid_acc)
    
    validation_loss_vs_epoch.append(valid_loss)
    
    #save the model if the validation loss has decreased
    if len(validation_loss_vs_epoch)==1 or validation_loss_vs_epoch[-2] > validation_loss_vs_epoch[-1]:
        torch.save(net.state_dict(), 'trained_model.pt')


  0%|                                                                                           | 0/50 [00:00<?, ?it/s]


RuntimeError: Given input size: (512x1x1). Calculated output size: (512x0x0). Output size is too small

for batch_x,batch_y in valid_dataloader:
  print(batch_x.shape, batch_y.shape)